In [6]:
import cv2
import numpy as np
import os
from tqdm import tqdm  # notebook 환경이 아닌 경우 tqdm.notebook 대신 tqdm 사용
from mtcnn import MTCNN

# MTCNN 탐지기 초기화
detector = MTCNN()

# 이미지가 있는 디렉토리
person_dir_path = "C:/Users/coldbrew/VTON-project/data/dataset/image/"
# 결과 이미지를 저장할 디렉토리
mosaiced_person_dir_path = "C:/Users/coldbrew/VTON-project/data/dataset/mosaiced_image/"

if not os.path.exists(mosaiced_person_dir_path):
    os.makedirs(mosaiced_person_dir_path)

person_file_list = os.listdir(person_dir_path)

for person_file in tqdm(person_file_list, total=len(person_file_list)):
    img_path = os.path.join(person_dir_path, person_file)
    img = cv2.imread(img_path)
    if img is None:
        print(f"이미지를 불러올 수 없습니다: {img_path}")
        continue
    
    # OpenCV는 BGR 순서를 사용하지만, MTCNN은 RGB를 사용하므로 변환
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # MTCNN으로 얼굴 검출
    faces = detector.detect_faces(img_rgb)

    for face in faces:
        x, y, w, h = face['box']
        
        # 이미지 범위를 벗어나는 경우를 방지하기 위해 좌표 보정
        x = max(0, x)
        y = max(0, y)
        w = max(0, w)
        h = max(0, h)
        
        face_roi = img[y:y+h, x:x+w]

        # 모자이크 스케일 조정
        mosaic_scale = 0.07  # 값이 작을수록 모자이크가 강해짐
        small = cv2.resize(face_roi, (0, 0), fx=mosaic_scale, fy=mosaic_scale, interpolation=cv2.INTER_LINEAR)
        mosaic_face = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)
        img[y:y+h, x:x+w] = mosaic_face

    # 결과 저장
    out_path = os.path.join(mosaiced_person_dir_path, person_file)
    cv2.imwrite(out_path, img)


100%|██████████| 195/195 [00:35<00:00,  5.52it/s]
